# 1. load the keypoints to fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.utils as fou
import os

# Assuming you have a list of image paths and corresponding TXT file paths
image_paths = [...]  # List of image file paths
txt_paths = [...]  # List of corresponding TXT file paths for each image

# Create a FiftyOne dataset
dataset = fo.Dataset("prawn_pose_estimation")
def parse_pose_estimation(txt_file):
    """
    Parse the pose estimation data from a TXT file.

    Parameters:
    txt_file (str): Path to the TXT file.

    Returns:
    list: A list of pose estimation data (each line parsed into a list of values).
    """
    pose_estimations = []
    with open(txt_file, 'r') as f:
        for line in f:
            pose_estimations.append([float(x) for x in line.strip().split()])
    return pose_estimations

# Loop over images and corresponding TXT files
for img_path, txt_path in zip(image_paths, txt_paths):
    # Parse the pose estimation data from the TXT file
    pose_estimations = parse_pose_estimation(txt_path)
    
    # List to hold all detections and keypoints for the current image
    detections = []
    
    for pose in pose_estimations:
        # Extract and scale bounding box and keypoints
        x_center_scaled = pose[1] * 5312
        y_center_scaled = pose[2] * 2988
        width_scaled = pose[3] * 5312
        height_scaled = pose[4] * 2988

        keypoints = []
        for i in range(5, len(pose), 3):
            x_kp_scaled = pose[i] * 5312
            y_kp_scaled = pose[i + 1] * 2988
            confidence = pose[i + 2]
            keypoints.append(fo.Keypoint(point=[x_kp_scaled, y_kp_scaled], confidence=confidence))

        bounding_box = [
            (x_center_scaled - width_scaled / 2) / 5312,
            (y_center_scaled - height_scaled / 2) / 2988,
            width_scaled / 5312,
            height_scaled / 2988
        ]

        # Create a detection object
        detection = fo.Detection(label="prawn", bounding_box=bounding_box, keypoints=keypoints)
        detections.append(detection)

    # Create a sample for FiftyOne
    sample = fo.Sample(filepath=img_path)
    sample["detections"] = fo.Detections(detections=detections)
    dataset.add_sample(sample)

# Launch the FiftyOne app to visualize
session = fo.launch_app(dataset)




# 2. load metadata from excel file

In [ ]:
import pandas as pd
import fiftyone as fo

# Read the Excel file
file_path = 'your_file_path.xlsx'  # Replace with your actual file path
metadata_df = pd.read_excel(file_path)

# Display the first few rows to ensure it's loaded correctly
print(metadata_df.head())


# Assuming you have a list of image paths
image_paths = [...]  # List of image file paths

# Create a FiftyOne dataset
dataset = fo.Dataset("prawn_metadata")

# Loop through each image and add metadata
for img_path in image_paths:
    # Extract the filename without extension
    filename = img_path.split('/')[-1].replace('.jpg', '')  # Adjust if not .jpg
    
    # Match the filename with the metadata
    metadata_row = metadata_df[metadata_df['file name'] == filename]
    
    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict()
        
        # Create a sample and attach metadata
        sample = fo.Sample(filepath=img_path)
        
        # Add each metadata field to the sample
        for key, value in metadata.items():
            if key != 'file name':  # Exclude the file name itself
                sample[key] = value
        
        # Add the sample to the dataset
        dataset.add_sample(sample)

# Launch the FiftyOne app to visualize
session = fo.launch_app(dataset)


# 3.bounding rectangle between detection and imagej

In [ ]:
import pandas as pd

# Load ImageJ bounding rectangles from a CSV file
imagej_bboxes_df = pd.read_csv('imagej_bboxes.csv')

# Example structure of `imagej_bboxes_df`:
# Columns: ['file name', 'x1', 'y1', 'x2', 'y2']
def calculate_bbox_distance(detection_bbox, imagej_bbox):
    """
    Calculate the Euclidean distance between the centers of two bounding boxes.

    Parameters:
    detection_bbox (tuple): (x1, y1, x2, y2) coordinates of the detection bounding box.
    imagej_bbox (tuple): (x1, y1, x2, y2) coordinates of the ImageJ bounding box.

    Returns:
    float: Euclidean distance between the centers of the bounding boxes.
    """
    # Calculate centers
    detection_center = ((detection_bbox[0] + detection_bbox[2]) / 2, (detection_bbox[1] + detection_bbox[3]) / 2)
    imagej_center = ((imagej_bbox[0] + imagej_bbox[2]) / 2, (imagej_bbox[1] + imagej_bbox[3]) / 2)
    
    # Calculate Euclidean distance between centers
    distance = np.sqrt((detection_center[0] - imagej_center[0])**2 + (detection_center[1] - imagej_center[1])**2)
    return distance
for img_path in image_paths:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    
    # Assume `detections` is a list of bounding boxes from the model
    detection_bboxes = [...]  # Replace with actual detection bounding boxes
    
    # Match with ImageJ bounding box
    imagej_bbox = imagej_bboxes_df[imagej_bboxes_df['file name'] == filename].iloc[0]
    imagej_bbox = (imagej_bbox['x1'], imagej_bbox['y1'], imagej_bbox['x2'], imagej_bbox['y2'])
    
    distances = []
    for detection_bbox in detection_bboxes:
        distance = calculate_bbox_distance(detection_bbox, imagej_bbox)
        distances.append(distance)
    
    # Create a FiftyOne sample and add metadata
    sample = fo.Sample(filepath=img_path)
    sample['bbox_distances'] = distances  # Store all distances for this image
    
    dataset.add_sample(sample)

# Launch FiftyOne app to visualize
session = fo.launch_app(dataset)


# 4. calculate length based on pinhole model

In [ ]:
import numpy as np

def calculate_real_width(focal_length, distance_to_object, width_in_pixels, pixel_size):
    """
    Calculate the real-life width of an object.

    Parameters:
    focal_length (float): Focal length of the camera lens in millimeters (mm).
    distance_to_object (float): Distance from the camera to the object in millimeters (mm).
    width_in_pixels (int): Width of the object in pixels on the image sensor.
    pixel_size (float): Size of a pixel on the image sensor in millimeters (mm).

    Returns:
    float: Real-life width of the object in centimeters (cm).
    """
    # Calculate the width of the object in the image sensor plane in millimeters
    width_in_sensor = width_in_pixels * pixel_size

    # Calculate the real-life width of the object using the similar triangles principle
    real_width_mm = (width_in_sensor * distance_to_object) / focal_length

    # Convert the width from millimeters to centimeters
    real_width_cm = real_width_mm / 10.0

    return real_width_cm

def calculate_euclidean_distance(keypoint1, keypoint2):
    """
    Calculate the Euclidean distance between two keypoints.

    Parameters:
    keypoint1, keypoint2: Tuples of (x, y) coordinates of the keypoints in pixels.

    Returns:
    float: Euclidean distance between the keypoints in pixels.
    """
    return np.sqrt((keypoint1[0] - keypoint2[0])**2 + (keypoint1[1] - keypoint2[1])**2)

# Example of how to integrate this with your FiftyOne dataset
for img_path in image_paths:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    metadata_row = metadata_df[metadata_df['file name'] == filename]
    
    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict()
        sample = fo.Sample(filepath=img_path)
        
        # Add metadata
        for key, value in metadata.items():
            if key != 'file name':
                sample[key] = value
        
        # Calculate the real height or width if keypoints and necessary metadata are available
        if 'height(mm)' in metadata:
            height_mm = metadata['height(mm)']
            focal_length = 6.82  # Example focal length in mm
            pixel_size = 0.0014  # Example pixel size in mm
            keypoints = [...]  # Replace with actual keypoints from your data
            
            if len(keypoints) >= 2:
                # Calculate the Euclidean distance in pixels
                euclidean_distance_pixels = calculate_euclidean_distance(keypoints[0], keypoints[1])
                
                # Calculate the real width/height in centimeters
                real_width_cm = calculate_real_width(focal_length, height_mm, euclidean_distance_pixels, pixel_size)
                
                # Attach the calculated real width/height to the sample
                sample["real_width_cm"] = real_width_cm
        
        dataset.add_sample(sample)

session = fo.launch_app(dataset)


# complete maybe

In [53]:
datasets = fo.list_datasets()

for dataset_name in datasets:
    if dataset_name.startswith('prawn_combined_dataset'):
        fo.delete_dataset(dataset_name)


Could not connect session, trying again in 10 seconds



In [64]:
import fiftyone as fo
import pandas as pd
import numpy as np
import os
import ast 
from tqdm import tqdm
import json


# Load the existing filtered data with PrawnIDs
filtered_data_file_path = r'C:\Users\gbo10\Videos\research\counting_research_algorithms\src\measurement\ImageJ\Filtered_Data.csv'  # Replace with your actual file path
filtered_df = pd.read_csv(filtered_data_file_path)

# Load the additional metadata from another Excel file
metadata_file_path = r"C:\Users\gbo10\OneDrive\research\thesis and paper\test images.xlsx"  # Replace with your actual file path
metadata_df = pd.read_excel(metadata_file_path)

# Function to parse pose estimation data from a TXT file
def parse_pose_estimation(txt_file):
    pose_estimations = []
    with open(txt_file, 'r') as f:
        for line in f:
            pose_estimations.append([float(x) for x in line.strip().split()])
    return pose_estimations

# Function to calculate Euclidean distance between keypoints
def calculate_euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Function to calculate the real-life width/height
def calculate_real_width(focal_length, distance_to_object, width_in_pixels, pixel_size):
    width_in_sensor = width_in_pixels * pixel_size
    real_width_mm = (width_in_sensor * distance_to_object) / focal_length
    return real_width_mm


folder_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\for fiftyone'  # Replace with the folder containing the images and TXT files

# Assuming you have the following lists
image_paths = [os.path.join(folder_path, image) for image in os.listdir(folder_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

label_folder_path=r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict18\labels"

txt_paths = [os.path.join(label_folder_path, txt) for txt in os.listdir(label_folder_path) if txt.endswith('.txt')]
# Create a FiftyOne dataset

import fiftyone as fo

# List all datasets in FiftyOne

# Loop through the list and delete datasets that match the condition


#delete all datasets


# Create a new dataset

dataset = fo.Dataset("prawn_combined_dataset8")

dataset.default_skeleton = fo.KeypointSkeleton(
    labels=[
        "start_carapace",
        "eyes",
    ],
    edges=[
        [0, 1],  # Connect keypoint 0 to keypoint 1  # Connect keypoint 1 to keypoint 2
        # Add more connections as needed
    ],
)
# Loop over images and corresponding TXT files
for img_path, txt_path in tqdm(zip(image_paths, txt_paths)):


    filename = os.path.splitext(os.path.basename(img_path))[0]
    
    parts = filename.split('_')
    relevant_part = f"{parts[1][-3:]}_{parts[3].split('.')[0]}"  # Extract the part like '152_378'
    


    # Parse the pose estimation data from the TXT file
    pose_estimations = parse_pose_estimation(txt_path)
    
    
    
    keypoints_list = []  # List to store keypoints
    detections = []  # List to store detection bounding boxes

    for pose in pose_estimations:
        if len(pose)==11:        

            x1_rel = pose[1] 
            y1_rel = pose[2] 
            width_rel = pose[3] 
            height_rel = pose[4] 

            
            #center to top left
            x1_rel = x1_rel - width_rel/2
            y1_rel = y1_rel - height_rel/2
        # Calc



            keypoints = []
            confidences = []
            for i in range(5, len(pose), 3):
                x_kp_scaled = pose[i] 
                y_kp_scaled = pose[i + 1] 
                confidence = pose[i + 2]
                keypoints.append([x_kp_scaled, y_kp_scaled])
                confidences.append(confidence)
            
            keypoint=fo.Keypoint(points=keypoints, confidence=confidences)
            keypoints_list.append(keypoint)


            keypoints_dict= {'point1':keypoints[i],'point2':keypoints[i+1]}
            detections.append(fo.Detection
                              (label="prawn", bounding_box=[x1_rel, y1_rel, width_rel, height_rel],
                               attributes={'keypoints':[keypoints_dict]
 }))

    sample = fo.Sample(filepath=img_path)
    
    # Add keypoints as a separate field
    sample["keypoints"] = fo.Keypoints(keypoints=keypoints_list)
   
    sample["detections"] = fo.Detections(detections=detections)
    


    
    
    # Match the filename with the corresponding rows in the filtered data
    matching_rows = filtered_df[filtered_df['Label'] == f'carapace:{filename}']
    
    # Add metadata from the additional Excel file
    metadata_row = metadata_df[metadata_df['file name'] == relevant_part]

    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict() 
        for key, value in metadata.items():
            if key != 'file name': 
                sample[key] = value
            
                        
    else:
        print(f"No metadata found for {relevant_part}")
    true_detections=[]

    for _, row in matching_rows.iterrows():
        prawn_id = row['PrawnID']
        prawn_bbox = ast.literal_eval(row['BoundingBox_1'])  # Replace with the correct column name

        # Convert the tuple elements to floats
        prawn_bbox = tuple(float(coord) for coord in prawn_bbox)# Replace with correct bounding box columns
        
        prawn_normalized_bbox=[prawn_bbox[0]/5312, prawn_bbox[1]/2988, prawn_bbox[2]/5312, prawn_bbox[3]/2988]

        # Convert PrawnID bounding box to absolute coordinates
        
        
        true_detections.append(fo.Detection(label="prawn_true", bounding_box=prawn_normalized_bbox))

        prawn_point=(prawn_bbox[0], prawn_bbox[1])
        

        # Find the closest detection bounding box within the same image (Label)
        min_distance = float('inf')
        closest_detection = None
        
        for detection_bbox in detections:
            detection_bbox_coords = detection_bbox.bounding_box
            
            det_point=(detection_bbox_coords[0], detection_bbox_coords[1])
            
            distance = calculate_euclidean_distance(prawn_point, det_point)
            
            if distance < min_distance:
                min_distance = distance
                closest_detection = detection_bbox
        
        if closest_detection is not None:
            height_mm = sample['heigtht(mm)']  # Use appropriate column for height in mm
            focal_length = 24.22
            pixel_size = 0.00716844 

                

            keypoints_dict2 = closest_detection.get_attribute_value('keypoints')

            keypoints = [keypoints_dict2['point1'], keypoints_dict2['point2']]        

            keypoint1_scaled = [keypoints[0][0] * 5312, keypoints[0][1] * 2988]
            keypoint2_scaled = [keypoints[1][0] * 5312, keypoints[1][1] * 2988]

            euclidean_distance_pixels = calculate_euclidean_distance(keypoint1_scaled, keypoint2_scaled)
            real_length_cm = calculate_real_width(focal_length, height_mm, euclidean_distance_pixels, pixel_size)
            

            # Update the filtered_df with the calculated lengths
            filtered_df.loc[(filtered_df['Label'] == f'carapace:{filename}') & (filtered_df['PrawnID'] == prawn_id), 'RealLength(cm)'] = real_length_cm
            
            # Add floating text near the keypoints in FiftyOne

            true_length=filtered_df.loc[(filtered_df['Label'] == f'carapace:{filename}') & (filtered_df['PrawnID'] == prawn_id), 'Avg_Length'].values[0]


            closest_detection_label =f"Real: {real_length_cm:.2f} cm, True: {true_length:.2f} cm, MPError: {abs(real_length_cm - true_length) / true_length * 100:.2f}%"
            
            closest_detection.label=closest_detection_label


            #if mpe>25: tag sample
            if abs(real_length_cm - true_length) / true_length * 100 > 25:
                if "MPE>25" not in sample.tags:
                    sample.tags.append("MPE>25")
            
    # Assuming 'sample' is your FiftyOne sample object
        
    sample["true_detections"] = fo.Detections(detections=true_detections)            

    dataset.add_sample(sample)
    

# Save the updated filtered data with the calculated lengths
# filtered_df.to_excel('updated_filtered_data_with_lengths.xlsx', index=False)

# Launch the FiftyOne app to visualize

# Define a custom view

# Configure the view to show your custom attributes

# Launch the app with the custom view
session = fo.launch_app(dataset)


0it [00:00, ?it/s]


AttributeError: 'list' object has no attribute 'items'


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds

